<a href="https://colab.research.google.com/github/ranjeeta01/100daysofcode/blob/master/code2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files

uploaded = files.upload()
import numpy as np
import csv

Saving feature_selection.py to feature_selection.py
Saving helper.py to helper.py
Saving nn.py to nn.py
Saving visualization.py to visualization.py


In [ ]:
import io  # for reading csv file uploaded in colab
import torch
from torch import nn
import pandas as pd
#from helper import read_csv

from prettytable import PrettyTable
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import time

from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.utils import to_categorical
from keras.callbacks import History
from keras.models import load_model

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def table_plot(data, col_labels, row_labels, title1, idx, accs):
    cols = np.append(np.append(idx, col_labels), 'Accuracy')
    data = np.array(data).astype(str)
    accs = np.array([accs])
    data = np.concatenate( (data, accs.T), axis=1)
    row_labels = np.array([row_labels])
    cells = np.concatenate( (row_labels.T, data), axis=1)
    x = PrettyTable(cols.tolist())
    for row in cells:
        x.add_row(row.tolist())
    print(x.get_string(title=title1))

In [ ]:
def print_database_information(y, y_train, y_val, y_test):

    _, counts_all   = np.unique(y, return_counts=True, axis=0)
    _, counts_train = np.unique(y_train, return_counts=True, axis=0)
    _, counts_val   = np.unique(y_val, return_counts=True, axis=0)
    _, counts_test  = np.unique(y_test, return_counts=True, axis=0)

    print("\n-----------------------------------")
    print("All Dataset:")
    print("-----------------------------------")
    for i in range(4):
        print("class", i, ': ', counts_all[i], '|', round(counts_all[i]/len(y)*100, 2), '%')
    print("-----------------------------------\n")

    print("-----------------------------------")
    print("Train Dataset:")
    print("-----------------------------------")
    for i in range(4):
        print("class", i, ': ', counts_train[i], '|', round(counts_train[i]/len(y_train)*100, 2), '%')
    print("-----------------------------------\n")

    print("-----------------------------------")
    print("Validation Dataset:")
    print("-----------------------------------")
    for i in range(4):
        print("class", i, ': ', counts_val[i], '|', round(counts_val[i]/len(y_val)*100, 2), '%')
    print("-----------------------------------\n")

    print("-----------------------------------")
    print("Test Dataset:")
    print("-----------------------------------")
    for i in range(4):
        print("class", i, ': ', counts_test[i], '|', round(counts_test[i]/len(y_test)*100, 2), '%')
    print("-----------------------------------\n")

In [ ]:
def execute_paper():
        #Train Neural Networks and Plot the image
        fig, axs = plt.subplots(nrows=1, ncols=2)
        titles  =['1:(5)', '1:(10)', '1:(100)', '2:(100x10)']
        colors = ['r', 'g', 'b', 'k']

        hidden_layers=[(5,), (10,), (100,), (100,10,)]
        neural_networks = [nn.create_mlp(11, hidden_layers[i], 4) for i in range(4)]

        earlyStopping = nn.MyEarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
        total_time = [0 for i in range(4)]

        for i in range(4):
            print(titles[i] + ' Start')

            curr_time = time.time()
            history = neural_networks[i].fit(X_train, y_train, epochs=500, batch_size=100, validation_data=(X_val, y_val), callbacks=[earlyStopping], verbose=0)

            total_time[i] = time.time() - curr_time
            optimal_point = earlyStopping.stopped_epoch

            #Plot:
            y_min = min(history.history["val_acc"])
            y_max = history.history["val_acc"][optimal_point]

            axs[0].plot(range(len(history.history["val_acc"])), history.history["val_acc"], color=colors[i], label=titles[i])
            axs[0].vlines(x=optimal_point, ymin=0.6, ymax=y_max, color='k', linestyle='--')
            legend = str(optimal_point) + ", " + str(round(history.history["val_acc"][-1]*100, 1)) + "%"
            axs[0].text(optimal_point+10, y_min+0.1, legend)#, ha='center', va='center',rotation='vertical')

            axs[0].set_xlabel("Epochs")
            axs[0].set_ylabel("Classification Accuracy")

            print(titles[i]+' Done!')
        axs[0].legend()

        def secs(x, pos):
            'The two args are the value and tick position'
            return '%1.1fs' % (x)
        formatter = FuncFormatter(secs)
        axs[1].yaxis.set_major_formatter(formatter)
        axs[1].bar(titles, total_time)

        plt.show()

In [ ]:
def print_confusion_matrix():

    tab_idx = ['a', 'b', 'c', 'd']
    classes = ['DP-BPSK', 'DP-QPSK', 'DP-16QAM', 'DP-64QAM']
    titles  =['1:(5)', '1:(10)', '1:(100)', '2:(100x10)']

    models = ["allFeatures_11-[5]-4", "allFeatures_11-[10]-4", "allFeatures_11-[100,10]-4", "allFeatures_11-[100]-4"]

    nothot_y_test = np.argmax(y_test, axis=1)
    for i in range(len(tab_idx)):

        clf = load_model("trained_models/" + models[i] + ".h5")

        y_pred = np.argmax(clf.predict(X_test), axis=1)
        acc = np.sum(y_pred==nothot_y_test)/len(nothot_y_test)

        accs = []
        for c in range(len(classes)):
            idx = nothot_y_test==c
            curr_acc = round(clf.evaluate(X_test[idx], y_test[idx], verbose=0)[1], 4)
            accs.append(curr_acc)

        cnf_matrix = confusion_matrix(nothot_y_test,y_pred)
        table_plot(cnf_matrix, classes, classes, titles[i], tab_idx[i]+')'+titles[i]+' Acc='+str(round(acc, 4)) , accs)

In [ ]:
if __name__ == "__main__":

        #Read data:
        X, y, features_name = pd.read_csv('data.csv')

        #Preprocessing:
        X = scale(X)
        y = to_categorical(y)

        #Split the dataset:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=20000, random_state=1)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=60000, random_state=1)

        #Database Information:
        #print_database_information(y, y_train, y_val, y_test)

        #Train Neural Networks and Plot the image
        #execute_paper()

        #Print table
        print_confusion_matrix()

ValueError: ignored

In [ ]:
C = 299792458

# lambda in m
LAMBDA = 1550e-9

# frequency in HZ
V = C/LAMBDA

# chosen OSNR noise BW in m
BN_db = 0.1 * 10**-9
BN = V**2 * BN_db # HZ

In [ ]:
THRESHOLD_OSNR = [12.6, 13, 16, 21] # OSNR_BPSK, OSNR_QPSK, OSNR_16QAM, OSNR_64QAM
MARGIN = [0.5, 1, 2, 3]             # MARGIN_BPSK, MARGIN_QPSK, MARGIN_16QAM, MARGIN_64QAM
CLASSIFICATION = [2, 4, 16, 64]     # class 0, class 1, class 2, class 3 | 4 is no_class

In [ ]:
class Simulator:

    def __init__(self):
        self.set_features()

    def set_features(self):

        self.symbol_rate     = np.random.randint(30, 91)
        self.roll_off        = np.random.randint(1, 6)/100
        self.channel_load    = np.random.randint(1, 121)
        self.dispersion      = np.random.uniform(4, 21)
        self.nonlinear_index = np.random.uniform(0.8, 1.6)
        self.loss            = np.random.uniform(0.15, 0.2)
        self.span_count      = np.random.randint(1, 51)
        self.span_length     = np.random.randint(40000, 120001)/1000
        self.noise_figure    = np.random.uniform(4, 6.5)

        self.status_linear   = False

    def get_features(self):

        features = [self.symbol_rate, self.roll_off, self.launch_power, self.channel_load, self.dispersion, self.nonlinear_index, self.loss, self.span_count, self.span_length, self.noise_figure, self.channel_grid]
        return features

    def convert_to_linear(self):

        if self.status_linear is True:
            return
        else:
            self.symbol_rate *= 1e9
            self.dispersion *= 1e-6
            self.nonlinear_index *= 1e-3
            self.loss = ((self.loss*1e-5)/4.34)*1e2
            self.span_length *= 1e3
            self.noise_figure = 10**(self.noise_figure/10)

            self.status_linear = True

    def calculate_osnr(self):

        #Defined Terms:
        gain = self.loss * self.span_length

        loss_eff = 1/(2*self.loss)
        self.channel_grid = (1 + self.roll_off) * self.symbol_rate
        bandwidth_wdm = self.channel_grid * self.channel_load

        #Noise Power
        noise_power =  H * V * BN * (gain * self.noise_figure - 1)

        #Launch Power:
        x = 2 * 16 * (self.nonlinear_index**2) * self.span_length *  BN * np.arcsinh(((1/3)*(np.pi**2))*self.dispersion*self.span_length*bandwidth_wdm**2)

        epsilon = 0.3 * np.log(1 + ((6*loss_eff)/(self.span_length*np.arcsinh(((np.pi**2)/2) * self.dispersion * loss_eff * bandwidth_wdm**2))))
        y = self.span_count**epsilon

        self.launch_power = self.symbol_rate * ((27 * np.pi * self.dispersion * noise_power) / (x*y))**(1/3)

        #OSNR:
        osnr_total = (2/3)*(self.launch_power/(noise_power*self.span_count))

        #Convert to dB:
        osnr_total_dB = 10 * np.log10(osnr_total)

        return osnr_total_dB

    def classify(self, osnr_total_dB):

        classification = 4
        for i in reversed(range(4)):
             L = osnr_total_dB - (THRESHOLD_OSNR[i])
             if 0 < L <  MARGIN[i]:
                return i
        return classification

    def simulate(self, nb_samples=1, output_path=None):

        all_samples = []
        while (len(all_samples) < nb_samples):

            #Set features
            self.set_features()
            self.convert_to_linear()

            #Simulate
            osnr_total_dB = self.calculate_osnr()

            #Classify
            classification = self.classify(osnr_total_dB)
            if classification == 4:
                continue

            #Data:
            features = self.get_features()
            data = [*features, classification]
            all_samples.append(data)

        with open(output_path, 'w', newline='') as csvfile:
            spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            spamwriter.writerow(['Symbol Rate', 'Roll-off', 'Launch Power', 'Channel load', 'Dispersion', 'Nonlinear index', 'Loss', 'Span count', 'Span lenght', 'Noise Figure', 'Channel grid', 'Modulation Classification'])
            spamwriter.writerows(all_samples)

In [ ]:
if __name__ == "__main__":

    simulator = Simulator()
    simulator.simulate(30000, output_path="data.csv")

NameError: ignored